<a href="https://colab.research.google.com/github/arashfahim/Deep-Schemes-for-Control/blob/main/Dr_Fahim_Second_Approach_(Final_Version).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive,files
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Importing

In [ ]:
import numpy as np
from scipy import misc
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, CyclicLR, ReduceLROnPlateau, LinearLR, ExponentialLR
import random
from torch.autograd.functional import jacobian, hessian
# import AUTOGRAD.FUNCTIONAL.JACOBIAN as jacobian
import time
import math

import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
mpl.rcParams['legend.fontsize'] = 10

# Hyperparameters

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Number of time intervals

In [ ]:
# num_time_interval=torch.tensor([10]).to(device)    #N
# start_list=[4,2,0]
# end_list=[6,8,10]

In [ ]:
num_time_interval=torch.tensor([20]).to(device)    #N
start_list=[4,3,2,1,0]
end_list=[6,7,8,9,10]

In [ ]:

# num_sample=torch.tensor([2**14]).to(device) #M
# dim=torch.tensor([3]).to(device)          #D

# T=torch.tensor([1]).to(device)
T=1
# T=torch.tensor([1])
delta_t = torch.tensor(T/ num_time_interval).to(device)
# t_steps = torch.linspace(0,1,num_time_interval).to(device)
# sqrt_delta_t = torch.sqrt(delta_t).to(device)
# sqrt_delta_t = torch.sqrt(delta_t).to(device)
sqrt_delta_t = torch.sqrt(delta_t).to(device)
# x_init = torch.zeros(dim)
# sigma=torch.sqrt(torch.tensor([2.0]))
# lamb=torch.tensor([1.0]).to(device)
mu=torch.tensor([1.0]).to(device)
sigma=torch.tensor([1.0]).to(device)
alpha=torch.tensor([1.0]).to(device)
start = torch.tensor([0]).to(device)
end = torch.tensor([1]).to(device)
num_steps=5
num_ite=int(num_time_interval/num_steps)
# kappa=torch.tensor([1]).to(device)
# theta=torch.tensor([0.4]).to(device)
# nu=torch.tensor([0.5]).to(device)
# lamb=torch.tensor([0.6]).to(device)
# eta=torch.tensor([0.5]).to(device)
p=torch.tensor([0.95]).to(device)
# rho=torch.tensor([0]).to(device)
# num_runs=3
# x0=torch.ones([1,dim]).to(device)+4
# print(x0)

t_steps = torch.linspace(0,T,num_time_interval.item()+1).to(device)
print(t_steps)

tensor([0.0000, 0.0500, 0.1000, 0.1500, 0.2000, 0.2500, 0.3000, 0.3500, 0.4000,
        0.4500, 0.5000, 0.5500, 0.6000, 0.6500, 0.7000, 0.7500, 0.8000, 0.8500,
        0.9000, 0.9500, 1.0000], device='cuda:0')


<ipython-input-6-d5c802e3e3eb>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  delta_t = torch.tensor(T/ num_time_interval).to(device)


##$n=1  \quad (d=2)$

In [ ]:
# dim=torch.tensor([2]).to(device)
# kappa=torch.tensor([1]).to(device)
# theta=torch.tensor([0.4]).to(device)
# nu=torch.tensor([1]).to(device)
# lamb=torch.tensor([0.3]).to(device)
# eta=torch.tensor([1]).to(device)
# rho=torch.tensor([0.0]).to(device)

##$n=2 \quad (d=3)$


In [ ]:
dim=torch.tensor([3]).to(device)
num_sample=torch.tensor([2**14]).to(device) #M
kappa=torch.tensor([1, 0.8]).to(device)
theta=torch.tensor([0.1,0.2]).to(device)
nu=torch.tensor([0.2, 0.15]).to(device)
lamb=torch.tensor([0.1, 0.15]).to(device)

# lamb=torch.tensor([0.1, 0.2, 0.3, 0.4, 0.5, 0.4, 0.3,0.2,0.1]).to(device)


eta=torch.tensor([1]).to(device)
rho=torch.tensor([0.0, 0.0]).to(device)

##$n=4 \quad (d=5) $

In [ ]:
# dim=torch.tensor([5]).to(device)
# num_sample=torch.tensor([2**16]).to(device) #M
# kappa=torch.tensor([1, 0.8, 1.1 ,1.3]).to(device)
# theta=torch.tensor([0.1,0.2,0.3,0.4]).to(device)
# nu=torch.tensor([0.2, 0.15, 0.25, 0.31]).to(device)
# lamb=torch.tensor([0.1, 0.15, 0.2,0.25]).to(device)
# eta=torch.tensor([1]).to(device)
# rho=torch.tensor([0.0, 0.0, 0.0, 0.0]).to(device)

##$n=9 \quad (d=10)$

In [ ]:
# dim=torch.tensor([10]).to(device)
# num_sample=torch.tensor([2**16]).to(device)
# kappa=torch.tensor([1, 0.8,1.1, 1.3, 0.95, 0.99,1.02, 1.06, 1.6]).to(device)
# theta=torch.tensor([0.1, 0.2, 0.3, 0.4, 0.25, 0.15, 0.18, 0.08, 0.91]).to(device)
# nu=torch.tensor([0.2, 0.15, 0.25, 0.2, 0.2, 0.1, 0.22, 0.2, 0.15]).to(device)

# lamb=torch.tensor([0.1, 0.125, 0.15, 0.175, 0.2, 0.225, 0.25,0.275,0.3]).to(device)
# # lamb=torch.tensor([0.1, 0.125, 0.15, 0.175, 0.2, 0.1, 0.12,0.12,0.12]).to(device)

# eta=torch.tensor([3]).to(device)
# rho=torch.tensor([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]).to(device)
# start_list=[5,3.5,3,1.5,0]
# end_list=[6,7.5,9,10.5,12]

# Functions

In [ ]:
def F(x,z,a): #### out_shape =([M]) |  input:  x_shape=[M,D,1],  z shape = [M,D,1],a_shape= [M,D,D]  #This is for rho=0
  result=torch.zeros([x.shape[0]]).to(device)
  for i in range(dim-1):
    result+=-kappa[i]*(theta[i]-x[:,i+1,0])*z[:,i+1,0]
    # print(result)
  result1=result-torch.sqrt(torch.sum(torch.square(lamb)))*torch.abs(z[:,0,0])*torch.sqrt(a[:,0,0])
  return result1
def H(x,z,gamma): #### out_shape =([M]) |  input:  x_shape=[M,D,1],  z shape = [M,D,1],gamma shape= [M,dim,dim]
  result=torch.zeros([x.shape[0]]).to(device)
  for i in range(dim-1):
    result+=kappa[i]*(theta[i]-x[:,i+1,0])*z[:,i+1,0]+0.5*torch.pow(nu[i],2)*gamma[:,i+1,i+1]-rho[i]*lamb[i]*nu[i]*(z[:,0,0]*gamma[:,0,i+1])/gamma[:,0,0] \
    + 0.5*torch.pow(rho[i]*nu[i]*gamma[:,0,i+1],2)/gamma[:,0,0]
    # print(result)
  result1=result-0.5*torch.sum(torch.square(lamb))*torch.pow(z[:,0,0],2)/gamma[:,0,0]
  return result1

def g1(x): #out_shape= [M,1,1]   | input:  x_shape=[M,D,1]
  # result=1-torch.exp(-eta*x[:,0])
  result=-torch.exp(-eta*x[:,0])
  result=torch.reshape(result,(result.shape[0],1,1))
  return result

def update(data,delta_w,L):#output: data=(M,D,1) ,  #input data=(M,D,1), delta_w=[M,D,1]
    dx = torch.bmm(L,delta_w)
    data = data + dx
    data=data
    return data

def der1(x,f):  #output= [M,D,1], input: x=[M,D,1]
  Du=torch.zeros(x.shape[0],dim).to(device)
  xin=x.clone().detach()
  xin.requires_grad=True
  u=f(xin)
  print(u)
  print('u shape', u.shape)
  Du=torch.autograd.grad(outputs=[u],inputs=[xin],grad_outputs=torch.ones_like(u),
                          allow_unused=True,retain_graph=True,create_graph=True)[0]
  print('Du shape', Du.shape)
  Du=torch.reshape(Du,(Du.shape[0],dim,1))
  print('Du shape', Du.shape)
  return Du

def grad_hessian1(x,f_,c=False): #output= [M,D,D], #input: x=[M,D,1]   #c=False => data=[M,D]   #c=True =>data=[M,D,1]
    hess_temp=torch.zeros(x.shape[0],dim,dim).to(device)
    Du=torch.zeros(x.shape[0],dim).to(device)

    if c==False:
      # xin=x.unsqueeze(2).clone().detach()
      xin=x.clone().detach()
      xin.requires_grad=True
      # print(xin.shape)
      # xin.requires_grad=True
      u=f_(xin).squeeze(2)
      # print(u.shape)
      # xin=xin.unsqueeze(2)
      # print(xin.shape)
      Du=torch.autograd.grad(outputs=[u],inputs=[xin],grad_outputs=torch.ones_like(u),
                           allow_unused=True,retain_graph=True,create_graph=True)[0].unsqueeze(2)
      hess_temp= torch.cat([ torch.autograd.grad(outputs=[Du[:,i,:]],inputs=[xin],grad_outputs=torch.ones_like(Du[:,i,:]),
                           allow_unused=True,retain_graph=True,create_graph=True)[0] for i in range(dim)],1)
    else:
      xin=x.clone().detach()
      xin.requires_grad=True
      u=f_(xin.squeeze(2))
    # print("xin shape: ",xin.shape)
      Du=torch.autograd.grad(outputs=[u],inputs=[xin],grad_outputs=torch.ones_like(u),
                           allow_unused=True,retain_graph=True,create_graph=True)[0]
    # print("Du shape:",Du.shape)
    # print("-----")
    # print(torch.autograd.grad(outputs=[Du[:,0,:]],inputs=[xin],grad_outputs=torch.ones_like(Du[:,0,:]),
    #                        allow_unused=True,retain_graph=True,create_graph=True)[0].shape)
      hess_temp= torch.cat([ torch.autograd.grad(outputs=[Du[:,i,:]],inputs=[xin],grad_outputs=torch.ones_like(Du[:,i,:]),
                           allow_unused=True,retain_graph=True,create_graph=True)[0] for i in range(dim)],1)
    # print("D2u shape:",hess_temp.shape)
    Du = torch.reshape(Du,(Du.shape[0],dim,1))
    # print("Du shape:",Du.shape)
    hess_temp=torch.reshape(hess_temp,(hess_temp.shape[0],dim,dim))
    # print("D2u shape:",hess_temp.shape)

    # print(hess_temp)
    return Du, hess_temp
def hessian1(x,model2z):  #x=[M,D,1]  output= [M,D,1]
    hess_temp=torch.zeros(x.shape[0],dim,dim).to(device)
    # print("D2u shape:",hess_temp.shape)

    Du=torch.zeros(x.shape[0],dim).to(device)
    xin=x.clone().detach()
    # print('this is xin')
    # print(xin)
    xin.requires_grad=True
    Du = model2z(xin.squeeze(2))
    hess_temp= torch.cat([ torch.autograd.grad(outputs=[Du[:,i,:]],inputs=[xin],grad_outputs=torch.ones_like(Du[:,i,:]),
                           allow_unused=True,retain_graph=True,create_graph=True)[0] for i in range(dim)],1)
    hess_temp=torch.reshape(hess_temp,(hess_temp.shape[0],dim,dim))

    return hess_temp


def V(t,x,v): #output           |#input:
  # result=1-torch.exp(-eta*x-0.5*(T-t)*torch.sum(torch.pow(lamb,2))).to(device)
  # result=1-torch.exp(-eta*x-0.5*(T-t)*torch.sum(torch.pow(lamb,2))).to(device)
  result=-torch.exp(-eta*x-0.5*(T-t)*torch.sum(torch.pow(lamb,2))).to(device)
  return result
def L_matrix(ite,x,model):  #x=[M,D]
  L=torch.zeros(x.shape[0],dim,dim).to(device)

  L[:,0,0]=model[ite](x)
  for i in range(dim-1):
    L[:,i+1,i+1]=nu[i]
  return L
def opt_quad():
  return torch.pow(torch.sum(torch.pow(lamb,2)),0.5)/eta

def L_matrix_test(ite,x,model1):  #This matrix is a matrix with L00 = 0.6 and L11=nu
  L=torch.zeros(x.shape[0],dim,dim).to(device)
  L[:,0,0]=opt_quad().to(device)
  for i in range(dim-1):
    L[:,i+1,i+1]=nu[i]
  return L

def gradient(x,f): #input=  [M,D,1]    #output= [M,D,1]
  xin=x.clone().detach()
  xin.requires_grad=True
  u=f(xin.squeeze(2));
  Du=torch.autograd.grad(outputs=[u],inputs=[xin],grad_outputs=torch.ones_like(u),allow_unused=True,retain_graph=True,create_graph=True)[0]
  return Du

## Plot Function

In [ ]:
def plot(ite,t1,steps, model1,model2,model11=False,model22=False):
  t_test=t1
  # xs = torch.linspace(start_list[0], end_list[0], steps=steps)
  # ys = torch.linspace(start_list[0],end_list[0], steps=steps)
  xs = torch.linspace(start_list[ite-1], end_list[ite-1], steps=steps)
  ys = torch.linspace(start_list[ite-1],end_list[ite-1], steps=steps)
  xgrid, ygrid = torch.meshgrid(xs, ys, indexing='xy')
  sss=torch.stack((xgrid.flatten(),ygrid.flatten()),1).to(device)
  print(sss.shape)

  # print(z1)

###############################################################################################################

  if model11==True:
    fig = plt.figure(figsize=plt.figaspect(0.5))
    # ax1 = fig.add_subplot(1, 2, 1, projection='3d')
    # z1=torch.abs(model1[ite-1](sss).clone().detach().cpu())
    z1=torch.abs(model1(sss).clone().detach().cpu())
    z1=torch.reshape(z1,(steps,steps))
    ax1=plt.axes(projection='3d')
    ax1.plot_surface(xgrid.numpy(), ygrid.numpy(), z1.numpy(),label = "trained_model1")
    lamb_value=torch.reshape(lamb/eta,(1,1)).cpu()
    lamb_matrix=np.ones((steps,steps))*lamb_value.numpy()
    # ax1=plt.axes(projection='3d')
    ax1.plot_surface(xgrid.numpy(), ygrid.numpy(), lamb_value.numpy())
    ax1.set_title("RMSE of the trained model at time step {}: {:.7f}".format(t_test,np.sqrt(np.mean(np.square(lamb_matrix-z1.numpy())))))
    ax1.set_xlabel(r'x', fontsize=15, rotation=10)
    ax1.set_ylabel(r'v', fontsize=15, rotation=10)
    # ax1.legend()
    # ax1.set_zlim(1.1, 1.3)
    plt.gcf().set_dpi(100)
    plt.show()

###########################################################################################################
  if model22==True:
    fig = plt.figure(figsize=plt.figaspect(0.5))
    # ax2 = fig.add_subplot(1, 2, 1, projection='3d')
    z2 = modely2[t_test](sss).clone().detach().cpu()
    # z2 = model2(sss).clone().detach().cpu()
    z2=torch.reshape(z2,(steps,steps))
    # print(z2)
    # print(z.shape)
    ax2 = plt.axes(projection='3d')
    ax2.plot_surface(xgrid.numpy(), ygrid.numpy(), z2.numpy(),label = "trained_modely2")

    truey = V(t_steps[t_test],xgrid.to(device),ygrid.to(device)).clone().detach().cpu()
    # print("This is theoretical solution")
    # print(truey)
    true_matrixy=torch.reshape(truey,(steps,steps))
    # print(true_matrix.shape)
    # ax2 = plt.axes(projection='3d')
    ax2.plot_surface(xgrid.numpy(), ygrid.numpy(), true_matrixy.numpy(),label = "True")


    # ax2.set_title("RMSE of the trained model Y at timestep {}: {:.7f}".format(t_test,np.sqrt(np.mean(np.square(true_matrixy.numpy()-z2.numpy())))))
    ax2.set_title("MSE of the trained model Y at timestep {}: {:.7f}".format(t_test,np.mean(np.square(true_matrixy.numpy()-z2.numpy()))))
    # ax2.set_xlabel(r'x', fontsize=15, rotation=60)
    # ax2.set_ylabel(r'v', fontsize=15, rotation=60)
    ax2.set_xlabel(r'x', fontsize=15, rotation=60)
    ax2.set_ylabel(r'v', fontsize=15, rotation=60)
    # ax2.set_zlim(-0.008, 0.004)
    # ax2.view_init(10, 10)
    plt.gcf().set_dpi(100)
    plt.show()


###########################################################################################################
#   ax = fig.add_subplot(1, 2, 2, projection='3d')
#   z3 = modelz2[t_test](sss).clone().detach().cpu()
#   trained_modelzx=z3.squeeze(2)[:,0].reshape(steps,steps)
#   # print(trained_modelzx)
#   trained_modelzv=z3.squeeze(2)[:,1].reshape(steps,steps)
#   print(trained_modelzv)
#   # # print(z.shape)
#   ax = plt.axes(projection='3d')
#   # ax.plot_surface(xgrid.numpy(), ygrid.numpy(), trained_modelzx.numpy(),label = "trained_modelzx")
#   ax.plot_surface(xgrid.numpy(), ygrid.numpy(), trained_modelzv.numpy(),label = "trained_modelzv")
#   truederx = derVx(t_steps[t_test],xgrid.to(device),ygrid.to(device)).clone().detach().cpu()
#   # print(truederx)
#   truederv=derVv(t_steps[t_test],xgrid.to(device),ygrid.to(device)).clone().detach().cpu()
#   print(truederv)
#   # ax.plot_surface(xgrid.numpy(), ygrid.numpy(), truederx.numpy(),label = "True derivative with respect to x")
#   ax.plot_surface(xgrid.numpy(), ygrid.numpy(), truederv.numpy(),label = "True derivative with respect to v")

# # ax3.set_title("MSE of the trained model Z at timestep {}: {:.7f}".format(t_test,np.mean(np.square(true_derx_matrix.numpy()-z2.numpy()))))



#   # ax.set_title("MSE of the trained model Y at timestep {}: {:.7f}".format(t_test,np.mean(np.square(true_matrix.numpy()-z3.numpy()))))
#   ax.set_xlabel(r'x', fontsize=15, rotation=70)
#   ax.set_ylabel(r'v', fontsize=15, rotation=60)
#   # ax.legend()

#   plt.gcf().set_dpi(100)
#   plt.show()

## Generating Data Functions

In [ ]:
def sample(ite): #output: [M,D]  #input: a scalar
    # x=(torch.rand(num_sample,1))
    # delta_w=torch.rand(num_sample,dim).to(device)*delta_t
    # x=(torch.rand(num_sample,1))
    if ite==num_ite:
      x=(start_list[ite-1])+(end_list[ite-1]-start_list[ite-1])*torch.rand(num_sample,dim)
      # x=(start_list[ite])+(end_list[ite]-start_list[ite])*delta_w
    else:
      x=(start_list[ite-1])+((end_list[ite-1])-(start_list[ite-1]))*torch.rand(num_sample,dim)
    return x

def data_generator2(ite,t1,t2,delta_w): #input: ite=scalar, t1=scalar, t2=scalar     #output: [M,D,1]
  count=0
  l=[]
  k=[]
  xx=torch.zeros([num_sample,dim,num_time_interval+1]).to(device)
  # xx=torch.zeros([num_sample,dim,t2-t1+2]).to(device)
  # xx[:,:,t1]=start_list[ite-1]+(end_list[ite-1]-start_list[ite-1])*(torch.rand(num_sample,dim))
  xx[:,:,t1]=start_list[ite-1]+(end_list[ite-1]-start_list[ite-1])*delta_w[:,:,t1-1]

  # if ite==num_ite:
  for i in range(t1, t2+1):
    L=L_matrix(ite-1,xx[:,:,i].to(device),model1)
    # L=L_matrix_test(ite-1,xx[:,:,i].to(device),model1)
    # print(L)
    xx[:,:,i+1]=(xx[:,:,i]+torch.bmm(L,delta_w[:,:,i].unsqueeze(2)).squeeze(2)).clone().detach()
  path_min = torch.min(xx[:,:,t1:t2+2].flatten(start_dim=1), dim=1)[0]
  # print(path_min)
  path_max = torch.max(xx[:,:,t1:t2+2].flatten(start_dim=1), dim=1)[0]
  # print(path_max)
  # pick_sample = torch.logical_and(path_max < end_list[ite]-2,path_min > start_list[ite]+2)
  pick_sample = torch.logical_and(path_max < end_list[ite],path_min > start_list[ite])
  # print('this is start_list[ite]',start_list[ite])
  # print('this is end_list[ite]',end_list[ite])
  # print(pick_sample)
  xx_new=xx[pick_sample,:,:]
  delta_w_new=delta_w[pick_sample,:,:]

  return xx_new.cpu(),delta_w_new.cpu()

## Remove Outlier Function

In [ ]:
def remove_outliers(ite,x):
    path_min = torch.min(x.flatten(start_dim=1), dim=1)[0]
    # print(path_min)
    path_max = torch.max(x.flatten(start_dim=1), dim=1)[0]
    # print(path_max)
    pick_sample = torch.logical_and(path_max < end_list[ite],path_min > start_list[ite])
    # print(pick_sample)
    x_new=x[pick_sample,:]
    # print('the percent of paths removed is {}%'.format((x.shape[0]-x_new.shape[0])/(x.shape[0])*100))
    return x_new

## Mean Error Function

In [ ]:
size=1000
test_samples=[(start_list[0]+(end_list[0]-start_list[0])*torch.rand(size).to(device)) for i in range(dim)]

def mean_error(t_test,ite,model):
  # test_samples=[(start+(end-start)*torch.rand(size).to(device)) for i in range(dim)]
  sss = torch.stack(test_samples,dim=1)
  z2=model[ite](sss).clone().detach().cpu()
  truey = V(t_test,sss,1).clone().detach().cpu()
  error=np.mean(np.square(truey.numpy()-z2.numpy()))
  return error

# def mean_error(size,start,end,t_test,ite,model):
#   test_samples=[(start+(end-start)*torch.rand(size).to(device)) for i in range(dim)]
#   sss = torch.stack(test_samples,dim=1)
#   z2=model[ite](sss).clone().detach().cpu()
#   truey = V(t_test,sss,1).clone().detach().cpu()
#   error=np.mean(np.square(truey.numpy()-z2.numpy()))
#   return error

# Initilizing Model 1

In [ ]:
class ann1(nn.Module):

    def __init__(self):
        super(ann1, self).__init__()
        self.linear_stack = nn.Sequential(

            nn.Linear(dim, 128),
            # nn.BatchNorm1d(256),
            nn.BatchNorm1d(num_features=128),
            nn.Tanh(),
            nn.Linear(128,128),
            nn.BatchNorm1d(num_features=128),
            # nn.BatchNorm1d(256),
            nn.Tanh(),
            # nn.Linear(256,256),
            # nn.Tanh(),
            nn.Linear(128, 128),
            nn.BatchNorm1d(num_features=128),
            # nn.BatchNorm1d(128),
            nn.Tanh(),
            # nn.Linear(128,128),
            # nn.Tanh(),
            # nn.Linear(128,128),
            # nn.Tanh(),
            # nn.Linear(128,128),
            # nn.Tanh(),
            nn.Linear(128,1),
        )
    def forward(self, x):
        logits = self.linear_stack(x)
        logits=torch.reshape(logits,(x.shape[0],))
        # logits=torch.flatten(logits)
        # logits=torch.reshape(logits,[x.shape[0]])
        # logits=nn.ELU()(logits)+1.5
        # logits=torch.reshape(logits,(x.shape[0],1,1))
        return torch.max(logits,-logits)

In [ ]:
model1= nn.ModuleList([ann1() for i in range(num_ite)]).to(device)

# Loss function 1

In [ ]:
######## TEST#######################
def loss_fn1(ite,x,delta_w,model):  #x=[M,D,1]  delta_w = [M,D,1]
    L=torch.zeros(x.shape[0],dim,dim).to(device)
    L[:,0,0]=model(x.squeeze(2))
    for i in range(dim-1):
      L[:,i+1,i+1]=nu[i]
    n=update(x,delta_w,L)
    if ite==num_ite:
      Y=g1(n).to(device)
      # print(Y.shape)
      # print("Y= ")
      # print(Y)
      Z,hessian=grad_hessian1(n,g1)
      # print("Z= ")
      # print(Z)
      # print("Hessian= ")
      # print(hessian)
    else:
      n=remove_outliers(ite,n)
      L=L[0:n.shape[0],:,:]
      Y=modely2[t2+1](n.squeeze(2))
      Z,hessian=grad_hessian1(n.squeeze(2),modely2[t2+1])
      Z=Z+0.001
      hessian=hessian+0.001
      # Y=modely2[t2+1](n.squeeze(2))
      # Z=modelz2[t2+1](n.squeeze(2))+0.0001
      # hessian=hessian1(n,modelz2[t2+1])+0.0001
      # print("Z= ", Z)
      # print("hessian= ",hessian)
    LTL=torch.bmm(torch.transpose(L,1,2),L)
    # print("LTL =", LTL)
    LTLhessian=torch.bmm(LTL,hessian)
    # print("LTLhessian =", LTLhessian)
    Tr=LTLhessian.diagonal(offset=0,dim1=-1,dim2=-2).sum(-1)
    # print("Trace =", Tr)
    H_=H(n,Z,hessian)
    # print("H = ", H_)
    F_=F(n,Z,LTL)
    # print("F= ", F_)
    # loss=torch.mean(torch.pow(H_-0.5*Tr+F_,2))
    loss=torch.mean(torch.pow(H_-0.5*Tr+F_,2))
    return loss

# Initializing Model 2

In [ ]:
class anny2(nn.Module): #input [M,D], #output [M,1,1]
    def __init__(self):
        super(anny2, self).__init__()
        self.linear_stack = nn.Sequential(
            nn.Linear(dim, 128),
            # nn.BatchNorm1d(256),
            # nn.BatchNorm1d(num_features=128),
            nn.Tanh(),
            nn.Linear(128,128),
            # nn.BatchNorm1d(num_features=128),
            nn.Tanh(),
            # nn.Linear(256, 256),
            # nn.Tanh(),
            nn.Linear(128,128),
            # nn.BatchNorm1d(num_features=128),
            nn.Tanh(),
            nn.Linear(128,128),
            # nn.BatchNorm1d(num_features=128),
            nn.Tanh(),
            # nn.Linear(128,128),
            # nn.Tanh(),
            # nn.Linear(128,128),
            # nn.Tanh(),
            # nn.Linear(128,128),
            # nn.Tanh(),
            nn.Linear(128,1),

        )
    def forward(self, x):
        logits = self.linear_stack(x)

        logits=torch.reshape(logits,(x.shape[0],1,1))
        return logits
        # return torch.min(logits,-logits)

class annz2(nn.Module): #input [M,D], #output [M,D,1]
    def __init__(self):
        super(annz2, self).__init__()
        self.linear_stack = nn.Sequential(
            nn.Linear(dim, 128),
            nn.BatchNorm1d(num_features=128),
            nn.Tanh(),
            nn.Linear(128,256),
            nn.BatchNorm1d(num_features=128),
            nn.Tanh(),
            nn.Linear(256, 256),
            nn.Tanh(),
            nn.Linear(256,128),
            # nn.Tanh(),
            # nn.Linear(128,128),
            # nn.Tanh(),
            # nn.Linear(128,128),
            # nn.Tanh(),
            # nn.Linear(128,128),
            # nn.Tanh(),
            # nn.Linear(128,128),
            # nn.Tanh(),
            # nn.Linear(128,128),
            # nn.Tanh(),
            # nn.Linear(128,128),
            # nn.Tanh(),
            nn.Linear(128,dim),
            # nn.ReLU(),
        )
    def forward(self, x):
        logits = self.linear_stack(x)

        logits=torch.reshape(logits,(x.shape[0],dim,1))
        # return logits
        return torch.min(logits,-logits)

In [ ]:
modely= anny2().to(device)
modelz= annz2().to(device)
modely2= nn.ModuleList([anny2() for i in range(num_time_interval+1)]).to(device)
modelz2= nn.ModuleList([annz2() for i in range(num_time_interval+1)]).to(device)

# Loss Function 2

In [ ]:
def loss_fn2(ite,data,delta_w):

  bb=L_matrix(ite-1,data[:,:,t1],model1).to(device)
  # bb=L_matrix_test(ite-1,data[:,:,t1],model1).to(device)
  ss=torch.pow(bb,2)
  loss=0
  Y=modely2[t1](data[:,:,t1])
  # print(data[:,:,t1].shape)
  # gradY,hessianY=grad_hessian1(data[:,:,t1],modely2[t1])
  # Z=modelz2[t1](data[:,:,t1])
  Z=gradient(data[:,:,t1].unsqueeze(2),modely2[t1]).to(device)
  # for ite in range(t1,t2):
  #   Y=Y.flatten()+F(data[:,:,ite].unsqueeze(2),Z,ss)*delta_t+torch.bmm(torch.bmm(torch.transpose(Z,1,2),ss),delta_w[:,:,ite].unsqueeze(2)).flatten()
  #   # bbb=L_matrix(num_ite-1,data[:,:,ite+1],model1)
  #   bbb=L_matrix_test(num_ite-1,data[:,:,ite+1],model1)
  #   ss=torch.pow(bbb,2)
  #   Z=modelz2[ite+1](data[:,:,ite+1])
  # Y=Y.flatten()+F(data[:,:,t2].unsqueeze(2),Z,ss)*delta_t+torch.bmm(torch.bmm(torch.transpose(Z,1,2),ss),delta_w[:,:,t2].unsqueeze(2)).flatten()

  # loss+=torch.mean(torch.square(Y-modely2[t2+1](data[:,:,t2+1]).flatten()))
  # # loss+=torch.mean(torch.square(Y-g1(data[:,:,t2+1]).flatten()))
  # return loss
  for i in range(t1,t2):
    Y=Y.flatten()+F(data[:,:,i].unsqueeze(2),Z,ss)*delta_t+torch.bmm(torch.bmm(torch.transpose(Z,1,2),bb),delta_w[:,:,i].unsqueeze(2)).flatten()
    bb=L_matrix(ite-1,data[:,:,i+1],model1).to(device)
    # bb=L_matrix_test(ite-1,data[:,:,i+1],model1).to(device)
    ss=torch.pow(bb,2)
    # Z=modelz2[i+1](data[:,:,i+1])
    Z=gradient(data[:,:,i+1].unsqueeze(2),modely2[i+1])
    # loss+=torch.mean(torch.mean(torch.square(gradient(data[:,:,i+1].unsqueeze(2),Y)-Z).squeeze(2),1))
    loss+=torch.mean(torch.square(Y-modely2[i+1](data[:,:,i+1]).flatten()))
  Y=Y.flatten()+F(data[:,:,t2].unsqueeze(2),Z,ss)*delta_t+torch.bmm(torch.bmm(torch.transpose(Z,1,2),bb),delta_w[:,:,t2].unsqueeze(2)).flatten()
  if ite==num_ite:
    loss+=torch.mean(torch.square(Y-g1(data[:,:,t2+1]).flatten()))
  else:
    loss+=torch.mean(torch.square(Y-modely2[t2+1](data[:,:,t2+1]).flatten()))
  return loss


#Training Both Model1 And Model2 Simultaneously

In [ ]:
from IPython.core.pylabtools import figsize
batch_size=int(num_sample/16)
num_epoch1 =100
num_epoch2=200
total_batch=int((num_sample/batch_size))
index=0
last_loss1 = 0
last_loss2=0
loss_delta1 = 0
loss_delta2=0
loss_treshold1 = 0
loss_treshold2=0
loss_tmp1=[]
loss_tmp2=[]
# my_index=torch.arange(num_sample)
# num_test_pts=500
num_runs=3

## Optimizer 1

In [ ]:
# lr1=[0.1, 0.1, 0.1, 0.1, 0.1]
# lr1=[0.000000015, 0.05, 0.01, 0.01, 0.01]
lr1=[0.001, 0.001, 0.001, 0.001, 0.001]
def opt1(ite, Adam=False,SGD=False):
  l=[]
  lr_=lr1[ite]

  if Adam==True:
    optimizer=optim.Adam(model1[ite-1].parameters(),lr=0.001)
  # optimizer=optim.Adam(model1[ite-1].parameters(),lr=lr_,weight_decay=1e-5)
  if SGD==True:
      optimizer=optim.SGD(model1[ite-1].parameters(), lr=lr_, momentum=0.9)
  # scheduler = ExponentialLR(optimizer, gamma =0.9)
  # optimizer = torch.optim.Adam(model1[ite-1].parameters(), lr=lr_, weight_decay=1e-5)

  return optimizer

## Optimizer 2

In [ ]:
lr2=[0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,0.001, 0.001, 0.001, 0.001, 0.001, 0.001]
# lr2=[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
# lr2=[0.015, 0.015, 0.015, 0.015, 0.015, 0.015, 0.015,0.015, 0.015, 0.015, 0.015, 0.015, 0.015]
# lr2=[0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,0.001, 0.001, 0.001, 0.001, 0.001, 0.001]
def opt2(ite,t1,t2,Adam=False,SGD=False):
  l=[]
  lr_=lr2[ite]

  # for i in range(t1,t2+1):
  #   l+=list(modely2[i].parameters())+list(modelz2[i].parameters())

  # l=list(modely2[t1].parameters())
  for i in range(t1,t2+1):
    l+=list(modely2[i].parameters())
  # optimizer=optim.Adam(l,lr=0.005)
  if Adam==True:
    optimizer=optim.Adam(l, lr=lr_)
  if SGD==True:
    optimizer=optim.SGD(l, lr=lr_, momentum=0.9)
  # optimizer = torch.optim.Adam(l, lr=lr_, weight_decay=1e-5)
  # scheduler = ExponentialLR(optimizer, gamma =0.9)
  return optimizer

## Value function
$u(t,x,v)=-e^{-\eta x-0.5(T-t)\lambda^2}$

## Train

In [ ]:
#### TESTTTTTTTTTT ##################
model1results=[]
modely2results=[]
modelz2results=[]
for i in range(num_runs): #numer of runs

  print('----------------------------------------------------------------------------------------------------------------------------------------')
  print("This is {}th run".format(i+1))
  model1= nn.ModuleList([ann1() for i in range(num_ite)]).to(device)
  modely2= nn.ModuleList([anny2() for i in range(num_time_interval+1)]).to(device)
  modelz2= nn.ModuleList([annz2() for i in range(num_time_interval+1)]).to(device)
  # delta_w1 = torch.randn(size=[num_sample, dim]).to(device) * sqrt_delta_t
  for ite in range(num_ite,0,-1):
    # model1results=[]
    # modely2results=[]
    # modelz2results=[]
    # "This is {}th run".format(i+1)
    print("this is ite=",ite)
    t_0 = time.time()
    # x0=torch.ones([1,dim]).to(device)+start_list[ite-1]
    # x01=torch.ones([100,dim]).to(device)+start_list[ite-1]
    # print("this is testing point x0",x0)
    # print("At iteration {}:".format( ite))
    t1=ite*num_steps-num_steps
    print("t1=", t1)
    t2=ite*num_steps-1
    print("t2=",t2)
    # print("this is t_test[t1]",t_steps[t1])
    # print("this is lambda", lamb)
    # print("this is eta", eta)
    # print("this is start_list[ite-1]+1", start_list[ite-1]+1)
    # print("this is T", T)
    # print("this is theortical solution", V(t_steps[t1],start_list[ite-1]+1,1))
    optimizer_1=opt1(ite,SGD=True)
    # if ite==num_ite:
    #   num_epoch1=150
    # else:
    #   num_epoch1=150
    # num_epoch1=0
    delta_w = torch.randn(size=[num_sample, dim]).to(device) * sqrt_delta_t
    for epoch in range(num_epoch1):

      # delta_w = torch.randn(size=[num_sample, dim]).to(device) * sqrt_delta_t
      data=sample(ite)
      my_index1=torch.arange(data.shape[0])
      total_batch1=int((data.shape[0]/batch_size))
      t_1=time.time()
      for batch in range(total_batch1):
        if batch==total_batch1-1:
              temp_index=my_index1[batch*batch_size : ]
        else:
              temp_index=my_index1[batch*batch_size : batch*batch_size + batch_size]

        optimizer_1.zero_grad()
        # loss1=loss_fn1(ite,data.unsqueeze(2)[temp_index,:].to(device),delta_w[temp_index,:,t2].unsqueeze(2).to(device),model1[ite-1])
        loss1=loss_fn1(ite,data.unsqueeze(2)[temp_index,:].to(device),delta_w[temp_index,:].unsqueeze(2).to(device),model1[ite-1])
        loss1.backward()
        optimizer_1.step()
      loss_tmp1.append(loss1.item())
      loss_delta1 = loss1.item() - last_loss1
      last_loss1 = loss1.item()
      if epoch %40 == 0:
        print("Loss function 1 at epoch {:.0f} is {:.12e}  |  Loss delta: {}   |    Training time: {:.5f}s".format(index*(num_epoch1)+epoch, loss_delta1,loss1.item(), time.time()-t_1))
        # print("Total time for training {:.0f} epochs is {:.4e}s".format(index*(num_epoch1)+epoch+1,time.time()-t_0))

    # print("The value of model1 at time step {} is {}".format(t2,torch.mean(model1[ite-1](x01)).item()))
    # print("The true optimal quadratic is {}".format(opt_quad().item()))
    # print("The  relative percentage error is {}: ".format((torch.abs((torch.mean(model1[ite-1](x01)))-opt_quad())).item()/torch.abs(model1[ite-1](x01)).item()))
    # index+=1
    # plot(ite,t1,100,model1[ite-1],modely2[t1],model11=True)

    optimizer_2=opt2(ite,t1,t2, SGD=True)
    for epoch in range(num_epoch2):
    # for epoch in range(100):
      t_2=time.time()
      delta_w = torch.randn(size=[num_sample, dim, num_time_interval]).to(device) * sqrt_delta_t
      # x_sample = torch.zeros([num_sample,dim, num_time_interval + 1])
      # x_sample[:, :, 0] = torch.zeros([num_sample, dim]) * x_init

      x_new,delta_w_new=data_generator2(ite,t1,t2,delta_w)
      # print('this is x_new',x_new.shape)
      my_index2=np.arange(x_new.shape[0])
      # print('this is my_index2',my_index2)
      total_batch2=int((x_new.shape[0]/batch_size))
      # print('this is batch_size',batch_size)
      # print('this is total_batch2',total_batch2)

      for batch in range(total_batch2):
        if batch==total_batch2-1:
              temp_index=my_index2[batch*batch_size : ]
        else:
              temp_index=my_index2[batch*batch_size : batch*batch_size + batch_size]
        data2=x_new[temp_index,:]
        # print(data2.shape)
        delta_w2=delta_w_new[temp_index,:]
        optimizer_2.zero_grad()
        loss2=loss_fn2(ite,data2.to(device),delta_w2.to(device))
        # print(loss2)
        loss2.backward()
        optimizer_2.step()
      loss_tmp2.append(loss2.item())
      loss_delta2 = loss2.item() - last_loss2
      last_loss2 = loss2.item()
      if epoch %40 == 0:
        print("Loss function 2 at epoch {:.0f} is {:.12e}  |  Loss delta: {}   |    Training time: {:.5f}s".format(index*(num_epoch2)+epoch, loss_delta2,loss2.item(), time.time()-t_2))
        # print("Total time for training {:.0f} epochs is {:.4e}s".format(index*(num_epoch1)+epoch+1,time.time()-t_0))
    index+=1

    # plot(ite,t1,100,model1[ite-1],modely2[t1],model22=True)

    # print("The value of modely2 at time step {} is {}".format(t1,modely2[t1](x0).item()))
    # print("The theoretical solution is {}".format(V(t_steps[t1],start_list[ite-1]+1,start_list[ite-1]+1).item()))
    # print("The relative percentage error is {}: ".format(torch.abs(modely2[t1](x0)-V(t_steps[t1],start_list[ite-1]+1,start_list[ite-1]+1)).item()/torch.abs(modely2[t1](x0)).item()))
  # model1results.append(model1[ite-1](x0))
  # modely2results.append(modely2[t1](x0))
    if ite==1:
      # x0=test_function(t1,dim,num_test_pts,start_list[t1],end_list[t1])
      # modely2results.append(test_function(t1,dim,num_test_pts,start_list[t1],end_list[t1]))
      # modely2results.append(mean_error(1000,start_list[0],end_list[0],0,0,modely2))
      modely2results.append(mean_error(0,0,modely2))
# print(model1results)
  print('this is accumulated mean square error from each run', modely2results)
# print("The average value of {} independent runs of modely2 at time step {} is {}".format(num_runs,t1,torch.mean(torch.tensor(model1results))))
print("The average value of {} independent runs of mse at time step {} is {}".format(num_runs,t1,torch.mean(torch.tensor(modely2results))))
print("The average std of {} independent runs of mse at time step {} is {}".format(num_runs,t1,torch.std(torch.tensor(modely2results))))

----------------------------------------------------------------------------------------------------------------------------------------
This is 1th run
this is ite= 4
t1= 15
t2= 19
Loss function 1 at epoch 0 is 2.158955794584e-06  |  Loss delta: 2.1589557945844717e-06   |    Training time: 1.19700s
Loss function 1 at epoch 40 is -4.244803903930e-07  |  Loss delta: 1.058130123965384e-06   |    Training time: 0.15348s
Loss function 1 at epoch 80 is 7.358539733104e-08  |  Loss delta: 1.6392505131079815e-06   |    Training time: 0.15440s
Loss function 2 at epoch 0 is 1.761971414089e-01  |  Loss delta: 0.1761971414089203   |    Training time: 0.19217s
Loss function 2 at epoch 40 is -4.375521093607e-03  |  Loss delta: 0.04109642654657364   |    Training time: 0.18787s
Loss function 2 at epoch 80 is -1.189118251204e-03  |  Loss delta: 0.031222598627209663   |    Training time: 0.18332s
Loss function 2 at epoch 120 is -7.436983287334e-04  |  Loss delta: 0.024899393320083618   |    Training ti

KeyboardInterrupt: ignored

In [ ]:
# x0=torch.ones([1,dim]).to(device)+4   #our testing region is [4,6] so x0=5
# print(x0)
# print(x0.shape)

In [ ]:
# aaa=[torch.rand(5)+5 for i in range(dim)]
# print(aaa)
# bbb = torch.stack(aaa,dim=1)  #50 randomly generated points for testing
# print(bbb)
# print(bbb.shape)
# print(bbb[:,0])
# ccc=modely2[0](bbb.to(device))  #evaluate these 50 random points using modely2 at time step 0
# print(ccc)
# print(ccc.shape)
# # ddd=V1(0,bbb[:,0].to(device))   #evaluate these 50 random point using true value function
# dddd=V(0,bbb[:,0].to(device),bbb[:,1].to(device))
# # print(ddd)
# print(dddd)
# eee=torch.mean(torch.square(ccc.flatten()-dddd))  #take square difference between true and approximated values
# print(eee)
# # print(V1(t_steps[t1],5.3743))
# # print(modely2[0](x0))

In [ ]:
# print(dim)
# print(start_list[0])
# print(start_list[0],end_list[0])
# print((start_list[0]+end_list[0])/2)
# # print(torch.mean(torch.tensor(3),torch.tensor(5)))

# SAVE (Path + Check Points)

In [ ]:
# PATH = "/content/gdrive//My Drive/'testsample.pt"
PATH = "/content/gdrive//My Drive/dim({})_num_sample({})_num_epoch({})_learning_rate1({})_learning_rate2({})_batchsize({}).pt".format(dim.item(),num_sample.item(),num_epoch2,lr1[1],lr2[1],batch_size)
torch.save({
            'epoch1': num_epoch1,
            'epoch2': num_epoch2,
            'model_state_dict1': model1[0].state_dict(),
            'model_state_dict2': modely2[0].state_dict(),
            'optimizer_state_dict1': optimizer_1.state_dict(),
            'optimizer_state_dict2': optimizer_2.state_dict(),
            # 'loss1':loss1.item(),
            # 'loss2': loss2.item(),
            'loss1_array': loss_tmp1,
            'loss2_array': loss_tmp2,
            'modely2results':modely2results,
            }, PATH)

In [ ]:
print(PATH)

# LOAD (Path + Check Points)

In [ ]:
checkpoint = torch.load(PATH)
loaded_model1=ann1().to(device)
loaded_model1.load_state_dict(checkpoint['model_state_dict1'])
loaded_model2=anny2().to(device)
loaded_model2.load_state_dict(checkpoint['model_state_dict2'])

loaded_optimizer1=opt1(1,SGD=True)
loaded_optimizer1.load_state_dict(checkpoint['optimizer_state_dict1'])


loaded_optimizer2=opt2(1,0,4,SGD=True)
loaded_optimizer2.load_state_dict(checkpoint['optimizer_state_dict2'])
epoch1 = checkpoint['epoch1']
# loss1 = checkpoint['loss1']
epoch2 = checkpoint['epoch2']
# loss2 = checkpoint['loss2']

loss1_array = checkpoint['loss1_array']
loss2_array = checkpoint['loss2_array']

modely2results=checkpoint['modely2results']

# Test

In [ ]:
print(modely2results)

In [ ]:
# plot(1,0,1000,loaded_model1,loaded_model2,model22=True)

In [ ]:
print(epoch2)
plt.plot(loss2_array)
plt.xlim(0, 499)

plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Function Value')
plt.show()